# Description
The objective of this analysis is to understand the benefits of the use of survival analysis for the modelling of credit risk lifetime default curves.
<br> The standard approach used in the industry is to:
1. Train a classification model in a single target, usually default in 12 months
2. Segment the portfolio in homogeneous risk groups.
3. Calibrate the PD in 12 months over each group.
4. calibrate the lifetime pd curve over each group.

<br> The approach described above breaks down each modelling component in different challenges, the scorecard is only worried with discrimination of the population between clients with low and high risk of defaulting while the calibration of the risk curves deal with both level and shape of the risk curves. This allows for a modular approach, although it implements additional steps in the modelling.
<br> An alternative approach would be using survival analysis methods which allows the modelling of the full lifetime curve in one single model.

This analysis aims to explore the use of survival models to calibrate pd lifetime risk curves

This notebook will create a syntetic set of data upon which the analysis will be done

# Setup

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy
from IPython.display import Image

In [2]:
pd.set_option('display.max_rows', 110)
output_path = "data/"

# Data Creation

In [3]:
def fit_power(x, shape, scale):
  return (scale * x ** (-shape))

def fit_exp(x, shape, scale):
  result = scale * np.exp(np.multiply(-shape, x))
  return result

def fit_weibull(x, shape, scale, d):
  result = (shape/scale)*np.power(((x+d)/scale), (shape-1)) * np.exp(-1 * np.power(((x+d)/scale), shape))
  return result

def fit_gamma(x, shape, scale, d=0):
  from scipy.special import gamma
  result = ((1 / (gamma(shape) * (scale ** shape))) * ((x+d) ** (shape - 1)) * np.exp(np.multiply(-1, (x+d)) / scale))
  return result

def logNormFunc(x, a, b, d):
  result = (1 / ((x+d) * b * np.power(2* np.pi, 1/2))) * np.exp(-1*(np.power(np.log(x+d) - a, 2) / (2*np.power(b, 2))))
  return result

def gamma_pdf(x, a, b):
  from scipy.special import gamma
  return (np.power(x, (a - 1)) * np.exp(np.divide(np.multiply(-1, x), b))) / (np.power(b, a) * gamma(a))

def exp_decay(x, a, b):
  return a * np.exp(np.multiply(-b, x))

def beta_pdf(x, a, b):
  from scipy.special import beta
  return np.divide((np.divide(x, np.power(60, (a - 1))) * np.power((1 - np.divide(x, 60)), (b - 1))), beta(a, b))

In [4]:
func_parameters =  dict(
    a = 1.5,
    b = 0.6,
    d = 0
)

risk_scores = {
    10 : 0.01,
     9 : 0.02,
     8 : 0.05,
     7 : 0.10,
     6 : 0.15,
     5 : 0.20,
     4 : 0.25,
     3 : 0.50,
     2 : 0.70,
     1 : 0.90
}

roll_rates = {
    30  :  0.75  , ## from 1-29    to 30-59
    60  :  0.90  , ## from 30-59   to 60-89
    90  :  0.99  , ## from 60-89   to 90-119
    120 :  0.99  , ## from 90-119  to 120-139
    150 :  0.999 , ## from 120-139 to 150-179
    180 :  0.999 , ## from 150-179 to 180-209
    210 :  0.999 , ## from 180-209 to 210-239
    240 :  0.999 , ## from 210-239 to 240-269
    270 :  0.999 , ## from 240-269 to 270-299
    300 :  0.999 , ## from 270-299 to 300-329
    330 :  0.999 , ## from 300-329 to 330-359
    360 :  0.9999, ## from 330-359 to 360-389
    390 :  1.0000, ## from 360-389 to 390-419
}

risk_scores_dist = {
    10 : 0.025,
     9 : 0.05,
     8 : 0.055,
     7 : 0.07,
     6 : 0.10,
     5 : 0.10,
     4 : 0.15,
     3 : 0.20,
     2 : 0.15,
     1 : 0.10
}

lifetime_target = 99

def generate_pd_curve(func, 
                      parameters: dict,
                      n_default_months: int,
                      cumulative_pd: float,
                      roll_rates: dict):
    df = pd.DataFrame(range(0, n_default_months+1), columns=["default_month"])
    # df['date'] = pd.date_range(start='2025-01-31', periods=n_default_months+1, freq='ME')
    df['default_month'] = df['default_month']
    df['p'] = func(df['default_month'], **parameters) 
    df['p'] = df['p'].fillna(0)
    df['p'] = (df['p'] / df['p'].sum()) * cumulative_pd
    df['cum_p'] = df['p'].cumsum() 
    multiplication_roll_rates = 1
    for days_late_i, roll_rate_i in roll_rates.items():
        if days_late_i <= 90:
            multiplication_roll_rates = multiplication_roll_rates * roll_rate_i
    df['pd_lifetime'] = (df['p'].sum() - df['cum_p']) * multiplication_roll_rates
    
    # Calculate forward-looking 12-month sum using rolling
    df['p_12m_forward'] = (df[::-1]['p'].rolling(window=12, min_periods=1, closed='left').sum().iloc[::-1]) * multiplication_roll_rates
    return df

def generate_pd_curve_by_risk_score(func,
                                    risk_scores: dict,
                                    parameters: dict,
                                    n_default_months: int,
                                    roll_rates: dict):
    result_dfs = list()
    for risk_band_i, lifetime_pd_i in risk_scores.items():
        parameters['a'] = risk_band_i / 2
        df_temp = generate_pd_curve(func, parameters, n_default_months, lifetime_pd_i, roll_rates)
        df_temp['risk_band'] = risk_band_i
        result_dfs.append(df_temp)
    result = pd.concat(result_dfs)
    return result



data = generate_pd_curve_by_risk_score(logNormFunc, risk_scores, func_parameters, lifetime_target, roll_rates)

data_chart = data.melt(id_vars=['default_month', 'risk_band'])

fig = px.line(
    data_chart,
    x='default_month',
    y='value',
    facet_col='variable',
    color='risk_band',
    template = 'none',
    width=1000
)
fig.update_yaxes(matches=None, showticklabels=True)
# Image(fig.to_image("png"))
fig

/home/neon/Documents/Dev/Survival Analysis/Survival-Analysis/.venv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/neon/Documents/Dev/Survival Analysis/Survival-Analysis/.venv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/neon/Documents/Dev/Survival Analysis/Survival-Analysis/.venv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/neon/Documents/Dev/Survival Analysis/Survival-Analysis/.venv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/neon/Documents/Dev/Survival Analysis/Survival-Analysis/.venv/lib/python3.10/si

In [5]:
data_chart = data.melt(id_vars=['default_month', 'risk_band'])

fig = px.line(
    data_chart,
    x='default_month',
    y='value',
    facet_col='risk_band',
    facet_col_wrap=4,
    color='variable',
    template = 'none',
    width=1000
)
fig.update_yaxes(matches=None, showticklabels=True)
# Image(fig.to_image("png"))
fig

In [69]:
start_date = "2025-01-31"
n_months = 24
n_sample = 10000
risk_scores_dist = {
    10 : 0.025,
     9 : 0.05,
     8 : 0.055,
     7 : 0.07,
     6 : 0.10,
     5 : 0.10,
     4 : 0.15,
     3 : 0.20,
     2 : 0.15,
     1 : 0.10
}

def generate_origination_sample(start_date: str,
                                n_months: int, 
                                n_sample: int,
                                risk_scores_dist: dict):
    id_list = range(1, n_sample+1)
    df_sample = pd.DataFrame(id_list, columns=['id'])
    df_sample['create_date_aux'] = np.random.randint(0,n_months, size=n_sample)
    df_sample['created_date'] = pd.to_datetime(pd.to_datetime(start_date) + df_sample['create_date_aux'].apply(lambda x : pd.offsets.MonthEnd(x)))
    df_sample['risk_band'] = np.random.choice(list(risk_scores_dist.keys()), size=n_sample, p=list(risk_scores_dist.values()))
    df_sample = df_sample.drop("create_date_aux", axis="columns")
    return df_sample

def withRiskCurves(df: pd.DataFrame, risk_curves: pd.DataFrame):
    result= df.merge(risk_curves, on=['risk_band'], how='left')
    result['date'] = pd.to_datetime(result['created_date'] + result['default_month'].apply(lambda x: pd.offsets.MonthEnd(int(x))))
    result = result.rename({'default_month':'tenure'}, axis='columns')
    return result

def withDefaultsFromRiskCurves(df: pd.DataFrame, roll_rates: dict):
    result = df.copy()
    result = result.sort_values(['id', 'date'])
    result['is_late'] = np.random.binomial(1, p=result['p'])
    result['late_start_date'] = pd.to_datetime(np.where(result['is_late'] == 1, result['date'], None))
    result['late_start_date'] = result.groupby('id')['late_start_date'].transform('min')
    result['days_late'] = np.where(result['late_start_date'] == result['date'], 1, None)
    for days_late_i, roll_rate_i in roll_rates.items():
        result['days_late_last_month'] = result.groupby('id')['days_late'].transform('shift', 1)
        result['prob_rr'] = roll_rate_i
        result['rgn'] = np.random.binomial(1, p=result['prob_rr'])
        result['days_late'] = np.where(result['days_late_last_month']==np.max([1, days_late_i-30]), 
                                       result['rgn'] *days_late_i, 
                                       result['days_late'])
        if days_late_i != list(roll_rates.keys())[-1]:
            result = result.drop(['days_late_last_month', 'rgn', 'prob_rr'], axis='columns')
    result['days_late'] = result.groupby('id')['days_late'].transform('ffill')
    result['days_late'] = result['days_late'].fillna(0)
    result['is_default'] = np.where(result['days_late'] == 90, 1, 0)
    result['n_late_status'] = result.groupby('id')['is_late'].transform('sum')
    return result

def withDefaultFlag(df: pd.DataFrame):
    result = df.copy()
    result['default_date_aux'] = pd.to_datetime(np.where(result['is_default']==1, result['date'], None))
    result = result.sort_values(['id', 'date'])
    result['default_date'] = result.groupby('id')['default_date_aux'].transform('min')
    result = result.drop('default_date_aux', axis='columns')
    result['default_month'] = pd.to_numeric(np.where(result['default_date'].isnull(), None,
                                       (result['default_date'].dt.to_period('M').astype(int) 
                                        - result['date'].dt.to_period('M').astype(int))))
    result['default_flag'] = np.where(result['default_month'].isnull(),0,1)
    return result

def filterAlreadyDefaulted(df: pd.DataFrame):
    filter_df = (df['default_month']>0) | (df['default_month'].isnull())
    result = df[filter_df]
    return result

def withCompleteDefaultMonths(df: pd.DataFrame, lifetime: int):
    default_months = pd.DataFrame(range(1,lifetime+1), columns=['complete_default_months'])
    id_df = df[['date', 'id']].drop_duplicates()\
                .merge(default_months, how='cross')
    
    result = id_df.merge(df, on=['date', 'id'], how='outer')
    result['default_flag'] = np.where(result['default_month'] == result['complete_default_months'], 1, 0)
    result = result\
                .drop('default_month', axis='columns')\
                .rename({'complete_default_months':'default_month'}, axis='columns')
    result['cum_defaults'] = result.sort_values(['date', 'id']).groupby(['date', 'id'])['default_flag'].transform('cumsum')
    return result

df_sample = generate_origination_sample(start_date, n_months, n_sample, risk_scores_dist)\
                    .pipe(withRiskCurves, data)\
                    .pipe(withDefaultsFromRiskCurves, roll_rates)\
                    .pipe(withDefaultFlag)\
                    .pipe(filterAlreadyDefaulted)\
                    .pipe(withCompleteDefaultMonths, 24)
df_sample



/tmp/ipykernel_295199/3937478368.py:31: PerformanceWarning:

Adding/subtracting object-dtype array to DatetimeArray not vectorized.

/tmp/ipykernel_295199/3937478368.py:52: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,date,id,default_month,created_date,risk_band,tenure,p,cum_p,pd_lifetime,p_12m_forward,...,late_start_date,days_late,days_late_last_month,prob_rr,rgn,is_default,n_late_status,default_date,default_flag,cum_defaults
0,2025-01-31,50,1,2025-01-31,7,0,0.000000e+00,0.0,6.682500e-02,0.003604,...,NaT,0,NaN,1.0,1,0,0,NaT,0,0
1,2025-01-31,50,2,2025-01-31,7,0,0.000000e+00,0.0,6.682500e-02,0.003604,...,NaT,0,NaN,1.0,1,0,0,NaT,0,0
2,2025-01-31,50,3,2025-01-31,7,0,0.000000e+00,0.0,6.682500e-02,0.003604,...,NaT,0,NaN,1.0,1,0,0,NaT,0,0
3,2025-01-31,50,4,2025-01-31,7,0,0.000000e+00,0.0,6.682500e-02,0.003604,...,NaT,0,NaN,1.0,1,0,0,NaT,0,0
4,2025-01-31,50,5,2025-01-31,7,0,0.000000e+00,0.0,6.682500e-02,0.003604,...,NaT,0,NaN,1.0,1,0,0,NaT,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19580515,2035-03-31,9927,20,2026-12-31,3,99,5.592856e-09,0.5,7.419065e-17,NaN,...,2027-04-30,0,None,1.0,1,0,1,NaT,0,0
19580516,2035-03-31,9927,21,2026-12-31,3,99,5.592856e-09,0.5,7.419065e-17,NaN,...,2027-04-30,0,None,1.0,1,0,1,NaT,0,0
19580517,2035-03-31,9927,22,2026-12-31,3,99,5.592856e-09,0.5,7.419065e-17,NaN,...,2027-04-30,0,None,1.0,1,0,1,NaT,0,0
19580518,2035-03-31,9927,23,2026-12-31,3,99,5.592856e-09,0.5,7.419065e-17,NaN,...,2027-04-30,0,None,1.0,1,0,1,NaT,0,0


In [67]:
df_sample[df_sample['default_flag']==1]

,id,created_date,risk_band,tenure,p,cum_p,pd_lifetime,p_12m_forward,date,is_late,late_start_date,days_late,days_late_last_month,prob_rr,rgn,is_default,n_late_status,default_date,default_month,default_flag
2200,23,2025-01-31,9,0,0.000000e+00,0.000000e+00,1.336500e-02,1.172002e-05,2025-01-31,0,2029-12-31,0,NaN,1.0,1,0,1,2030-03-31,62.0,1
2201,23,2025-01-31,9,1,1.432904e-14,1.432904e-14,1.336500e-02,1.837520e-05,2025-02-28,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,61.0,1
2202,23,2025-01-31,9,2,2.129409e-11,2.130842e-11,1.336500e-02,2.750916e-05,2025-03-31,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,60.0,1
2203,23,2025-01-31,9,3,8.224241e-10,8.437325e-10,1.336500e-02,3.960661e-05,2025-04-30,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,59.0,1
2204,23,2025-01-31,9,4,8.330965e-09,9.174698e-09,1.336499e-02,5.514957e-05,2025-05-31,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,58.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999695,9997,2026-10-31,2,95,1.168966e-10,7.000000e-01,2.373540e-10,2.373539e-10,2034-09-30,0,2027-02-28,390,None,1.0,1,0,1,2027-05-31,-88.0,1
999696,9997,2026-10-31,2,96,1.043024e-10,7.000000e-01,1.676539e-10,1.676538e-10,2034-10-31,0,2027-02-28,390,None,1.0,1,0,1,2027-05-31,-89.0,1
999697,9997,2026-10-31,2,97,9.314719e-11,7.000000e-01,1.054083e-10,1.054082e-10,2034-11-30,0,2027-02-28,390,None,1.0,1,0,1,2027-05-31,-90.0,1
999698,9997,2026-10-31,2,98,8.325713e-11,7.000000e-01,4.977169e-11,4.977165e-11,2034-12-31,0,2027-02-28,390,None,1.0,1,0,1,2027-05-31,-91.0,1


In [68]:
df_sample[df_sample['id']==23]

,id,created_date,risk_band,tenure,p,cum_p,pd_lifetime,p_12m_forward,date,is_late,late_start_date,days_late,days_late_last_month,prob_rr,rgn,is_default,n_late_status,default_date,default_month,default_flag
2200,23,2025-01-31,9,0,0.000000e+00,0.000000e+00,1.336500e-02,0.000012,2025-01-31,0,2029-12-31,0,NaN,1.0,1,0,1,2030-03-31,62.0,1
2201,23,2025-01-31,9,1,1.432904e-14,1.432904e-14,1.336500e-02,0.000018,2025-02-28,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,61.0,1
2202,23,2025-01-31,9,2,2.129409e-11,2.130842e-11,1.336500e-02,0.000028,2025-03-31,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,60.0,1
2203,23,2025-01-31,9,3,8.224241e-10,8.437325e-10,1.336500e-02,0.000040,2025-04-30,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,59.0,1
2204,23,2025-01-31,9,4,8.330965e-09,9.174698e-09,1.336499e-02,0.000055,2025-05-31,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,58.0,1
2205,23,2025-01-31,9,5,4.284925e-08,5.202394e-08,1.336497e-02,0.000075,2025-06-30,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,57.0,1
2206,23,2025-01-31,9,6,1.473956e-07,1.994195e-07,1.336487e-02,0.000098,2025-07-31,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,56.0,1
2207,23,2025-01-31,9,7,3.897964e-07,5.892159e-07,1.336461e-02,0.000127,2025-08-31,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,55.0,1
2208,23,2025-01-31,9,8,8.580760e-07,1.447292e-06,1.336403e-02,0.000160,2025-09-30,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,54.0,1
2209,23,2025-01-31,9,9,1.651746e-06,3.099038e-06,1.336293e-02,0.000198,2025-10-31,0,2029-12-31,0,None,1.0,1,0,1,2030-03-31,53.0,1


In [70]:
data_chart = df_sample\
    .groupby(['date','default_month'])\
    .agg({
        'id' : 'count',
        'cum_defaults': 'mean',
        'default_flag': 'mean'
    }) \
    .reset_index()\
    .melt(id_vars=['date', 'default_month'])


fig = px.line(
    data_chart,
    x='default_month',
    y='value',
    facet_col='variable',
    color='date',
    template='none',
    width=1000
)

fig.update_yaxes(matches=None, showticklabels=True)
# Image(fig.to_image("png", width=1000))


In [8]:
df_sample.to_parquet(f"{output_path}syntetic_sample.parquet")